In [1]:
import requests
import boto3
import json
import os
import sys
import argparse
import importlib
import transformers
import torch
import pathlib
from IPython.display import display
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role

# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
config.py loaded: v0.1
utils.py loaded: v0.2.12
utils.py loaded: v0.2.12
config.py loaded: v0.1


In [3]:
import sagemaker
sagemaker.__version__

'2.250.0'

In [4]:
import torch
torch.__version__

'2.5.1+cu124'

In [5]:
import transformers
transformers.__version__

'4.49.0'

In [18]:
removed = """
S3_INPUT_DIR_NAME = '00_test/input'
S3_OUTPUT_DIR_NAME = '00_test/output'

S3_INPUT_DIR_PATH = f'{config.DEFAULT_S3_BUCKET_ROOT}/{S3_INPUT_DIR_NAME}'
S3_OUTPUT_DIR_PATH = f'{config.DEFAULT_S3_BUCKET_ROOT}/{S3_OUTPUT_DIR_NAME}'



#Initialize the HuggingFaceProcessor
hfp = HuggingFaceProcessor(
    role=get_execution_role(), 
    instance_count=1,
    py_version='py311',
    transformers_version=transformers.__version__,
    pytorch_version=torch.__version__.split('+')[0],  # Get the version before any suffix like "+cpu"
    instance_type='ml.g5.2xlarge',
    base_job_name='test-job-002',
    # max_runtime_in_seconds=86400
)

#Run the processing job
hfp.run(
    code='02_ingestion/02_02_ingestion_semanticscholar.py',
    source_dir=os.path.abspath(os.path.join(os.getcwd(), '..')),
    inputs=[
        ProcessingInput(
            input_name='data',
            source=S3_INPUT_DIR_PATH,
            destination=f'{config.DEFAULT_PROCESSING_FILEPATH_PREFIX}/input/data/'
        )
    ],
    outputs=[
        ProcessingOutput(output_name='results', source=f'{config.DEFAULT_PROCESSING_FILEPATH_PREFIX}/output/results/', destination=S3_OUTPUT_DIR_PATH),
        # ProcessingOutput(output_name='train', source='/opt/ml/processing/output/train/', destination=f's3://{BUCKET}/{S3_OUTPUT_PATH}'),
        # ProcessingOutput(output_name='test', source='/opt/ml/processing/output/test/', destination=f's3://{BUCKET}/{S3_OUTPUT_PATH}'),
        # ProcessingOutput(output_name='val', source='/opt/ml/processing/output/val/', destination=f's3://{BUCKET}/{S3_OUTPUT_PATH}')
    ],
    arguments=[
        '--runtype', 'prod',
        '--test-argument-key-01', 'test-argument-value-01', 
        '--test-argument-key-02', 'test-argument-value-02'
    ]
)
"""

In [ ]:
removed = """
S3_INPUT_DIR_NAME = '00_test/input'
S3_OUTPUT_DIR_NAME = '00_test/output'

S3_INPUT_DIR_PATH = f'{config.DEFAULT_S3_BUCKET_ROOT}/{S3_INPUT_DIR_NAME}'
S3_OUTPUT_DIR_PATH = f'{config.DEFAULT_S3_BUCKET_ROOT}/{S3_OUTPUT_DIR_NAME}'

# Using FrameWorkProcessor primarily to be able to use source_dir during the run step, even if we don't need the full flexibility of FrameWorkProcessor or the SKLearn framework
execution_role = get_execution_role()
# pipeline_session = PipelineSession()
print(f'Using execution role: {execution_role}')
sklearn_processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version='1.2-1', # The newest supported version by sagemaker
    instance_type='ml.t3.medium', # We are just downloading&uploading data, so a small instance is sufficient
    instance_count=1,
    base_job_name='SemanticsScholar-ingestion-FrameworkProcessor-SKlearn-1-7-0',
    # sagemaker_session=pipeline_session,
    role=execution_role
)

step_args = sklearn_processor.run(
    code='02_ingestion/02_02_ingestion_semanticscholar.py',
    source_dir=os.path.abspath(os.path.join(os.getcwd(), '..')),
    inputs=[
        ProcessingInput(
            input_name='data',
            source=S3_INPUT_DIR_PATH,
            destination=f'{config.DEFAULT_PROCESSING_FILEPATH_PREFIX}/input/data/'
        )
    ],
    outputs=[
        ProcessingOutput(output_name='results', source=f'{config.DEFAULT_PROCESSING_FILEPATH_PREFIX}/output/results/', destination=S3_OUTPUT_DIR_PATH),
        # ProcessingOutput(output_name='train', source='/opt/ml/processing/output/train/', destination=f's3://{BUCKET}/{S3_OUTPUT_PATH}'),
        # ProcessingOutput(output_name='test', source='/opt/ml/processing/output/test/', destination=f's3://{BUCKET}/{S3_OUTPUT_PATH}'),
        # ProcessingOutput(output_name='val', source='/opt/ml/processing/output/val/', destination=f's3://{BUCKET}/{S3_OUTPUT_PATH}')
    ],
    arguments=[
        '--runtype', 'prod',
        '--process-files-force-overwrite', 'False',
        '--test-argument-key-01', 'test-argument-value-01', 
        '--test-argument-key-02', 'test-argument-value-02'
    ]
)

# step_process = ProcessingStep(
#     name="ProcessingNameTODO",
#     step_args=step_args
# )

# #Initialize the HuggingFaceProcessor
# hfp = HuggingFaceProcessor(
#     role=get_execution_role(), 
#     instance_count=1,
#     py_version='py311',
#     transformers_version=transformers.__version__,
#     pytorch_version=torch.__version__.split('+')[0],  # Get the version before any suffix like "+cpu"
#     instance_type='ml.g5.2xlarge',
#     base_job_name='test-job-002',
#     # max_runtime_in_seconds=86400
# )

#Run the processing job
# sklearn_processor.run(
#     code='02_ingestion/02_02_ingestion_semanticscholar.py',
#     source_dir=os.path.abspath(os.path.join(os.getcwd(), '..')),
#     inputs=[
#         ProcessingInput(
#             input_name='data',
#             source=S3_INPUT_DIR_PATH,
#             destination=f'{config.DEFAULT_PROCESSING_FILEPATH_PREFIX}/input/data/'
#         )
#     ],
#     outputs=[
#         ProcessingOutput(output_name='results', source=f'{config.DEFAULT_PROCESSING_FILEPATH_PREFIX}/output/results/', destination=S3_OUTPUT_DIR_PATH),
#         # ProcessingOutput(output_name='train', source='/opt/ml/processing/output/train/', destination=f's3://{BUCKET}/{S3_OUTPUT_PATH}'),
#         # ProcessingOutput(output_name='test', source='/opt/ml/processing/output/test/', destination=f's3://{BUCKET}/{S3_OUTPUT_PATH}'),
#         # ProcessingOutput(output_name='val', source='/opt/ml/processing/output/val/', destination=f's3://{BUCKET}/{S3_OUTPUT_PATH}')
#     ],
#     arguments=[
#         '--runtype', 'prod',
#         '--test-argument-key-01', 'test-argument-value-01', 
#         '--test-argument-key-02', 'test-argument-value-02'
#     ]
# )

"""

In [ ]:
dataset_id = 's2orc'
# Using FrameWorkProcessor primarily to be able to use source_dir during the run step, even if we don't need the full flexibility of FrameWorkProcessor or the SKLearn framework
execution_role = get_execution_role()
sklearn_processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version='1.2-1', # The newest supported version by sagemaker
    instance_type='ml.t3.medium', # We are just downloading&uploading data, so a small instance is sufficient
    instance_count=1,
    base_job_name=f'semanticsScholar-ingestion-{dataset_id}',
    role=execution_role
)

step_args = sklearn_processor.run(
    code='02_ingestion/02_11_ingestion_semanticscholar.py',
    source_dir=os.path.abspath(os.path.join(os.getcwd(), '..')),
    inputs=[], # We are not using automatic input-output mapping, instead we handle everything in the script directly on S3
    outputs=[],
    arguments=[
        '--runtype', 'prod',
        '--dataset-id', dataset_id,
        '--release-id', '2025-08-12',
        # '--force-overwrite', '0',
        '--min-index', '0',
        '--max-index', '1000000',
    ]
)

INFO:sagemaker.processing:Uploaded /home/sagemaker-user/research_methodology_extraction/src to s3://sagemaker-eu-west-2-762595428873/SemanticsScholar-ingestion-2025-08-23-19-44-07-718/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-2-762595428873/SemanticsScholar-ingestion-2025-08-23-19-44-07-718/source/runproc.sh
INFO:sagemaker:Creating processing-job with name SemanticsScholar-ingestion-2025-08-23-19-44-07-718


..............utils.py loaded: v0.2.12
config.py loaded: v0.1
 :: :: TIMELOGGER STARTED :: | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
Keys stored in Secrets Managers for the secret "semanticscholar_api_key": ['x-api-key']
 :: Semanticscholar secret keys fetched | since_start: 0.18 seconds | since_last: 0.18 seconds :: 
Namespace(runtype='prod', dataset_id='s2orc', release_id='2025-08-12', force_overwrite=False, min_index=0, max_index=1000000)
 :: Processed arguments | since_start: 0.18 seconds | since_last: 0.00 seconds :: 
File already exists in S3: s3://sagemaker-research-methodology-extraction/01_data/01_raw/semanticscholar/s2orc/s2orc-part0.jsonl.gz. Skipping download and upload.
File already exists in S3: s3://sagemaker-research-methodology-extraction/01_data/01_raw/semanticscholar/s2orc/s2orc-part1.jsonl.gz. Skipping download and upload.
File already exists in S3: s3://sagemaker-research-methodology-extraction/01_data/01_raw/semanticscholar/s2orc/s2orc-part2.jsonl

In [16]:
execution_role = get_execution_role()
default_semanticscholar_framework_processor_params = {
    'estimator_cls': SKLearn,
    'py_version': 'py3',
    'framework_version': '1.2-1', # The newest supported version by sagemaker
    'instance_type': 'ml.t3.medium', # We are just downloading&uploading data, so a small instance is sufficient
    'instance_count': 1,
    'role': execution_role
}

default_semanticscholar_processor_run_params = {
    'code': 'src/02_ingestion/02_11_ingestion_semanticscholar.py',
    'source_dir': os.path.abspath(os.path.join(os.getcwd(), '..', '..')),
    'inputs': [], # We are not using automatic input-output mapping, instead we handle everything in the script directly on S3
    'outputs': [],
    'wait': False
}

default_semanticscholar_processor_run_arguments = [
    '--runtype', 'prod',
    '--release-id', '2025-08-12',
    '--min-index', '0',
    '--max-index', '1000000',
]

default_semanticscholar_processor_run_params

{'code': 'src/02_ingestion/02_11_ingestion_semanticscholar.py',
 'source_dir': '/home/sagemaker-user/research_methodology_extraction',
 'inputs': [],
 'outputs': [],
 'wait': False}

In [15]:
dataset_id = 'papers'
# Using FrameWorkProcessor primarily to be able to use source_dir during the run step, even if we don't need the full flexibility of FrameWorkProcessor or the SKLearn framework

sklearn_processor = FrameworkProcessor(
    **default_semanticscholar_framework_processor_params,
    base_job_name=f'semanticsScholar-ingestion-{dataset_id.replace("_", "-")}',
)

step_args = sklearn_processor.run(
    **default_semanticscholar_processor_run_params,
    arguments=list(default_semanticscholar_processor_run_arguments + ['--dataset-id', dataset_id])
)

INFO:sagemaker.processing:Uploaded /home/sagemaker-user/research_methodology_extraction to s3://sagemaker-eu-west-2-762595428873/semanticsScholar-ingestion-papers-2025-08-24-21-49-36-889/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-2-762595428873/semanticsScholar-ingestion-papers-2025-08-24-21-49-36-889/source/runproc.sh
INFO:sagemaker:Creating processing-job with name semanticsScholar-ingestion-papers-2025-08-24-21-49-36-889


.............CodeArtifact repository not specified. Skipping login.
Found existing installation: typing 3.7.4.3
Uninstalling typing-3.7.4.3:
  Successfully uninstalled typing-3.7.4.3
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of pathos to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 37.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 97.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 114.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 10.2 MB/s  0:01:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   base_job_name=f'semanticsScholar-ingestion-{dataset_id.replace("_", "-")}',             │
│    7 )                                                                                           │
│    8                                                                                             │
│ ❱  9 step_args = sklearn_processor.run(                                                          │
│   10 │   **default_semanticscholar_processor_run_params,                                         │
│   11 │   arguments=list(default_semanticscholar_processor_run_arguments + ['--dataset-id', da    │
│   12 )                                                                                           │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/python_311/lib/python3.11/site-packages/sagemaker/workflow/pipe │
│ line_context.py:346 in wrapper                                                                   │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/python_311/lib/python3.11/site-packages/sagemaker/processing.py │
│ :1780 in run                                                                                     │
│                                                                                                  │
│   1777 │   │   )                                                                                 │
│   1778 │   │                                                                                     │
│   1779 │   │   # Submit a processing job.                                                        │
│ ❱ 1780 │   │   return super().run(                                                               │
│   1781 │   │   │   code=s3_runproc_sh,                                                           │
│   1782 │   │   │   inputs=inputs,                                                                │
│   1783 │   │   │   outputs=outputs,                                                              │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/python_311/lib/python3.11/site-packages/sagemaker/workflow/pipe │
│ line_context.py:346 in wrapper                                                                   │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)             

In [17]:
dataset_id = 's2orc_v2'
# Using FrameWorkProcessor primarily to be able to use source_dir during the run step, even if we don't need the full flexibility of FrameWorkProcessor or the SKLearn framework

sklearn_processor = FrameworkProcessor(
    **default_semanticscholar_framework_processor_params,
    base_job_name=f'semanticsScholar-ingestion-{dataset_id.replace("_", "-")}',
)

step_args = sklearn_processor.run(
    **default_semanticscholar_processor_run_params,
    arguments=list(default_semanticscholar_processor_run_arguments + ['--dataset-id', dataset_id])
)

INFO:sagemaker.processing:Uploaded /home/sagemaker-user/research_methodology_extraction to s3://sagemaker-eu-west-2-762595428873/semanticsScholar-ingestion-s2orc-v2-2025-08-24-22-02-21-096/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-2-762595428873/semanticsScholar-ingestion-s2orc-v2-2025-08-24-22-02-21-096/source/runproc.sh
INFO:sagemaker:Creating processing-job with name semanticsScholar-ingestion-s2orc-v2-2025-08-24-22-02-21-096


In [20]:
dataset_id = 'paper-ids'
# Using FrameWorkProcessor primarily to be able to use source_dir during the run step, even if we don't need the full flexibility of FrameWorkProcessor or the SKLearn framework

sklearn_processor = FrameworkProcessor(
    **default_semanticscholar_framework_processor_params,
    base_job_name=f'semanticsScholar-ingestion-{dataset_id.replace("_", "-")}',
)

step_args = sklearn_processor.run(
    **default_semanticscholar_processor_run_params,
    arguments=list(default_semanticscholar_processor_run_arguments + ['--dataset-id', dataset_id])
)

INFO:sagemaker.processing:Uploaded /home/sagemaker-user/research_methodology_extraction to s3://sagemaker-eu-west-2-762595428873/semanticsScholar-ingestion-paper-ids-2025-08-24-22-39-44-582/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-2-762595428873/semanticsScholar-ingestion-paper-ids-2025-08-24-22-39-44-582/source/runproc.sh
INFO:sagemaker:Creating processing-job with name semanticsScholar-ingestion-paper-ids-2025-08-24-22-39-44-582


In [19]:
dataset_id = 'abstracts'
# Using FrameWorkProcessor primarily to be able to use source_dir during the run step, even if we don't need the full flexibility of FrameWorkProcessor or the SKLearn framework

sklearn_processor = FrameworkProcessor(
    **default_semanticscholar_framework_processor_params,
    base_job_name=f'semanticsScholar-ingestion-{dataset_id.replace("_", "-")}',
)

step_args = sklearn_processor.run(
    **default_semanticscholar_processor_run_params,
    arguments=list(default_semanticscholar_processor_run_arguments + ['--dataset-id', dataset_id])
)

INFO:sagemaker.processing:Uploaded /home/sagemaker-user/research_methodology_extraction to s3://sagemaker-eu-west-2-762595428873/semanticsScholar-ingestion-abstracts-2025-08-24-22-13-31-596/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-west-2-762595428873/semanticsScholar-ingestion-abstracts-2025-08-24-22-13-31-596/source/runproc.sh
INFO:sagemaker:Creating processing-job with name semanticsScholar-ingestion-abstracts-2025-08-24-22-13-31-596


In [ ]:
glue_crawler_semanticscholar_processor_run_params = {
    'code': 'src/02_ingestion/02_12_ingestion_semanticscholar_glue_crawler.py',
    'source_dir': os.path.abspath(os.path.join(os.getcwd(), '..', '..')),
    'inputs': [], # We are not using automatic input-output mapping, instead we handle everything in the script directly on S3
    'outputs': [],
    'wait': True,
    'arguments': [
        '--runtype', 'prod',
    ]
}


# Using FrameWorkProcessor primarily to be able to use source_dir during the run step, even if we don't need the full flexibility of FrameWorkProcessor or the SKLearn framework
sklearn_processor = FrameworkProcessor(
    **default_semanticscholar_framework_processor_params,
    base_job_name=f'semanticsScholar-glue-crawler',
)

step_args = sklearn_processor.run(
    **glue_crawler_semanticscholar_processor_run_params
)

In [ ]:
import awswrangler as wr

openalex_useless_tables = [
'openalex_updated_date_2024_01_15openalex_updated_date_2024_02_03',
'openalex_updated_date_2024_02_04',
'openalex_updated_date_2024_02_05',
'openalex_updated_date_2024_02_06',
'openalex_updated_date_2024_02_07',
'openalex_updated_date_2024_02_19',
'openalex_updated_date_2024_02_20',
'openalex_updated_date_2024_02_21',
'openalex_updated_date_2024_03_16',
'openalex_updated_date_2024_03_19',
'openalex_updated_date_2024_03_19_9e79c0714477582ebc4d277347f48ce3',
'openalex_updated_date_2024_03_20',
'openalex_updated_date_2024_03_22',
'openalex_updated_date_2024_03_24',
'openalex_updated_date_2024_03_25',
'openalex_updated_date_2024_05_16',
'openalex_updated_date_2024_05_17',
'openalex_updated_date_2024_06_28',
'openalex_updated_date_2024_06_30',
'openalex_updated_date_2024_06_30_3349cf01e46eb64a0084560c35563ade',
'openalex_updated_date_2024_07_07',
'openalex_updated_date_2024_07_10',
'openalex_updated_date_2024_08_06',
'openalex_updated_date_2024_08_10_7fec41323b16ec8a2a2da85d87135850',
'openalex_updated_date_2024_08_11_f7a005d2518c1ec677333d5a1d03ff0a',
'openalex_updated_date_2024_08_12',
'openalex_updated_date_2024_08_13_945135d76fae9ab1022dbdc4e2584c73',
'openalex_updated_date_2024_08_18',
'openalex_updated_date_2024_08_18_29ead1d8af7c46c730ec8290afc517d3',
'openalex_updated_date_2024_09_02_7797cd1566a044b97b2ea2e0fbea32b4',
'openalex_updated_date_2024_09_03',
'openalex_updated_date_2024_09_04',
'openalex_updated_date_2024_09_05',
'openalex_updated_date_2024_09_06',
'openalex_updated_date_2024_09_12_ba475ef2f8808f7eb9862678ebf668e3',
'openalex_updated_date_2024_09_16_55a6e7ab256b115ea4b71c9b42a68329',
'openalex_updated_date_2024_09_17_502ca502150a5ee1cf763c77823f2686',
'openalex_updated_date_2024_09_22',
'openalex_updated_date_2024_10_04',
'openalex_updated_date_2024_10_05',
'openalex_updated_date_2024_10_06',
'openalex_updated_date_2024_10_07',
'openalex_updated_date_2024_10_08_04001e693ce5936bbafb0e6ef7573c9e',
'openalex_updated_date_2024_10_09_341d76ac46ea9d3a3b337d9e413a0375',
'openalex_updated_date_2024_10_13',
'openalex_updated_date_2024_11_11',
'openalex_updated_date_2024_11_13',
'openalex_updated_date_2024_11_14',
'openalex_updated_date_2024_11_15',
'openalex_updated_date_2024_11_17',
'openalex_updated_date_2024_11_18',
'openalex_updated_date_2024_11_19_0886fd50bff656bb7f19f6d4b7dca5e2',
'openalex_updated_date_2024_11_21_4c3201c76b0a3d4e49d9deabc5fda429',
'openalex_updated_date_2024_11_23',
'openalex_updated_date_2024_11_24',
'openalex_updated_date_2024_11_24_0e74be6753767804610dd310a09083fe',
'openalex_updated_date_2024_11_25',
'openalex_updated_date_2024_11_26',
'openalex_updated_date_2024_11_27',
'openalex_updated_date_2024_12_01',
'openalex_updated_date_2024_12_02',
'openalex_updated_date_2024_12_02_457b339f60afc53feef12a91f3c49f42',
'openalex_updated_date_2024_12_03',
'openalex_updated_date_2024_12_12_ae9aee316f7390edcdd525ffa28914a0',
'openalex_updated_date_2024_12_13',
'openalex_updated_date_2024_12_13_09972473b1483fd3c2995c1a4275f13d',
'openalex_updated_date_2024_12_14',
'openalex_updated_date_2024_12_15',
'openalex_updated_date_2024_12_16',
'openalex_updated_date_2024_12_16_1a1b2ef9049f79e07a03eb27ba8ff8c8',
'openalex_updated_date_2024_12_26',
'openalex_updated_date_2024_12_26_0a85f05a26afdfaba2397e1e3f8ec3b1',
'openalex_updated_date_2024_12_27',
'openalex_updated_date_2025_01_14',
'openalex_updated_date_2025_01_15',
'openalex_updated_date_2025_01_31',
'openalex_updated_date_2025_02_15',
'openalex_updated_date_2025_02_16',
'openalex_updated_date_2025_02_17',
'openalex_updated_date_2025_02_18',
'openalex_updated_date_2025_02_19',
'openalex_updated_date_2025_02_20',
'openalex_updated_date_2025_02_21',
'openalex_updated_date_2025_02_22',
'openalex_updated_date_2025_02_23',
'openalex_updated_date_2025_03_01',
'openalex_updated_date_2025_03_01_86a2cc0c73d0689bda5dc759881cca0c',
'openalex_updated_date_2025_03_02_e315adfb31aab603d19090e97fedaa4a',
'openalex_updated_date_2025_03_04_9f162c7c7a3d78af107fc53d462eb045',
'openalex_updated_date_2025_03_06_a4cd7bc0e77fab8d1b5da5f2b6ef09b9',
'openalex_updated_date_2025_03_09_95cd4c3cc8baec51bbd7741a3c8e6b19',
'openalex_updated_date_2025_03_12_e761b2dc95ba1b427dc7846a37cf577d',
'openalex_updated_date_2025_03_24_3a1de3b644812635e6febf25a9dbc868',
'openalex_updated_date_2025_03_25_aae51a293067d16ecf07ea4f29eb2ae6',
'openalex_updated_date_2025_03_26',
'openalex_updated_date_2025_03_26_7e69edb8b798aefe11af85367838b001',
'openalex_updated_date_2025_03_27',
'openalex_updated_date_2025_03_27_dae3f9045cfbff8e2d83aee7292a5fa6',
'openalex_updated_date_2025_03_28',
'openalex_updated_date_2025_03_28_0ddf4780c715d02fa3592f746fde8688',
'openalex_updated_date_2025_03_29',
'openalex_updated_date_2025_03_29_d5f6efe4ebe64d945ffdabb843ee67cf',
'openalex_updated_date_2025_03_30',
'openalex_updated_date_2025_03_31',
'openalex_updated_date_2025_04_02',
'openalex_updated_date_2025_04_03',
'openalex_updated_date_2025_04_09_bca312cc17f541ca7cf448e614753b89',
'openalex_updated_date_2025_04_10_c53c272ec248a709321eceb8ef37980d',
'openalex_updated_date_2025_04_17',
'openalex_updated_date_2025_04_18',
'openalex_updated_date_2025_04_19',
'openalex_updated_date_2025_04_20',
'openalex_updated_date_2025_04_21',
'openalex_updated_date_2025_04_22',
'openalex_updated_date_2025_04_23',
'openalex_updated_date_2025_04_28',
'openalex_updated_date_2025_04_29',
'openalex_updated_date_2025_04_30',
'openalex_updated_date_2025_05_01',
'openalex_updated_date_2025_05_02',
'openalex_updated_date_2025_05_03',
'openalex_updated_date_2025_05_04',
'openalex_updated_date_2025_05_05',
'openalex_updated_date_2025_05_06',
'openalex_updated_date_2025_05_11',
'openalex_updated_date_2025_05_12',
'openalex_updated_date_2025_05_13',
'openalex_updated_date_2025_05_26_f94efc7f8932c9fd6d3572356672aa5a',
'openalex_updated_date_2025_05_28_c1be9e2e418a307cd1f3faf3d2158215',
'openalex_updated_date_2025_06_28',
'openalex_updated_date_2025_07_11',
'openalex_updated_date_2025_07_12',
'openalex_updated_date_2025_07_20_ed7873e3af43ee9911a8ecb4fb803788',
'openalex_updated_date_2025_07_21',
'openalex_updated_date_2025_07_22',
'openalex_updated_date_2025_07_23',
'openalex_updated_date_2025_07_23_54d296cb8182c9c99b41a3e3f382e9ea',
'openalex_updated_date_2025_07_26_16d09c46cc17964ad4a4865222b38fcc',
'openalex_updated_date_2025_07_27_07c4d269b4196d034b7bc68fcd35c979',
'openalex_updated_date_2025_07_28_49bb02c97c72548d23369750e4330070',
'openalex_updated_date_2025_07_29',
'openalex_updated_date_2025_08_01',
'openalex_updated_date_2025_08_02',
'openalex_updated_date_2025_08_04',
'openalex_updated_date_2025_08_05_8d3002a57d1c1bfb699968785c8bc028',
'openalex_updated_date_2025_08_06_c09210a84316fd535165dc07c6e444b7',
'openalex_updated_date_2025_08_07_de8e5a66fc6997966f3def5e6a0b671e',
'openalex_updated_date_2025_08_11_5184d1682444bf6a9d86cd8d99252fce',
'openalex_updated_date_2025_08_12_c346cd8bab3c2c382720e21378de1b6e',
'openalex_updated_date_2025_08_13_242dc4eee8875de2e354bfb635901127',
'openalex_updated_date_2025_08_14',
'openalex_updated_date_2025_08_15',
'openalex_updated_date_2025_08_15_e35fb1a138634d73eb2975ee3b58597d',
'openalex_updated_date_2025_08_16',
'openalex_updated_date_2025_08_17',
'openalex_updated_date_2025_08_20',
'openalex_updated_date_2025_08_21_e1d125afdad531b69e25f8994f5de578',
]
# Delete tables from athena
for table in openalex_useless_tables:
    wr.catalog.delete_table_if_exists(
        database='01_raw',
        table=table
    )
time_logger = utils.TimeLogger()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:162                                                                                  │
│                                                                                                  │
│   159 ]                                                                                          │
│   160 # Delete tables from athena                                                                │
│   161 for table in openalex_useless_tables:                                                      │
│ ❱ 162 │   utils.delete_athena_table_if_exists(                                                   │
│   163 │   │   database_name='01_raw',                                                            │
│   164 │   │   table_name=table,                                                                  │
│   165 │   │   aws_region=config.AWS_REGION                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: module 'utils' has no attribute 'delete_athena_table_if_exists'